In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from geopy.distance import geodesic
import numpy as np
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [241]:
shike = pd.read_excel('C:\intern\shike.xlsx')
shike_sorted = shike.sort_values(by=['路线', '车次id', '站序'])

In [242]:
shike_df = shike_sorted.copy()

In [243]:
shike_df = shike_df.drop(['经度','纬度'], axis = 1)

In [244]:
shike_df

,路线,车次id,站序,站点,时间,环路
2303,Y1,411,1,临江,07:40:00,False
2304,Y1,411,2,云驰街,07:46:00,False
2305,Y1,412,1,临江,07:50:00,False
2306,Y1,412,2,云驰街,07:56:00,False
2307,Y1,413,1,临江,07:55:00,False
...,...,...,...,...,...,...
685,Y9,132,5,行知路,16:25:00,False
686,Y9,132,6,光电科技园,16:27:00,False
687,Y9,132,7,耀华,16:30:00,False
688,Y9,132,8,网易,16:31:00,False


In [245]:
checi_all = shike_df['车次id'].unique()

In [246]:
shike_df[shike_df['车次id']==411]

,路线,车次id,站序,站点,时间,环路
2303,Y1,411,1,临江,07:40:00,False
2304,Y1,411,2,云驰街,07:46:00,False


In [247]:
shike_df[shike_df['车次id']==2]

,路线,车次id,站序,站点,时间,环路
10,Y4,2,1,临江,07:40:00,False
11,Y4,2,2,云驰街,07:45:00,False
12,Y4,2,3,中信建设,07:50:00,False
13,Y4,2,4,网易,07:54:00,False
14,Y4,2,5,耀华,07:55:00,False
15,Y4,2,6,光电科技园,07:56:00,False
16,Y4,2,7,膜园,07:58:00,False


In [248]:
type(shike_df[(shike_df['站点']=='临江') & (shike_df['车次id']==411)]['时间'].values[0])

str

In [249]:
def time_check(current_checi, other_checi, zhandian):
    current_shijian = shike_df[(shike_df['站点']==zhandian) & (shike_df['车次id']==current_checi)]['时间'].values[0]
    other_shijian = shike_df[(shike_df['站点']==zhandian) & (shike_df['车次id']==other_checi)]['时间'].values[0]
    t1 = datetime.strptime(current_shijian, "%H:%M:%S")
    t2 = datetime.strptime(other_shijian, "%H:%M:%S")
    interval = float('inf')
    res =  False
    if t1>t2:
        interval = (t1-t2).total_seconds()
    else:
        interval = (t2-t1).total_seconds()
    
    if interval <300:
        res = True
        
    return res

In [250]:
def find_other_checi_with_zhandian(current_checi, zhandian, df):
    other_checi_all = df[df['站点']==zhandian]['车次id'].values
    other_checi_all = other_checi_all.tolist()
    if current_checi in other_checi_all:
        other_checi_all.remove(current_checi)
    other_checi_time_valid  = []
    duiying_zhanxu = []
    
    for each in other_checi_all:
        if time_check(current_checi, each, zhandian):
            other_checi_time_valid.append(each)
            
            duiying_zhanxu.append(df[(df['车次id']==each)&(df['站点']==zhandian)]['站序'].values[0])
    
    return other_checi_time_valid,duiying_zhanxu

In [251]:
def find_checi_with_left_zhandian(current_checi, zhandian, df, start_zhandian_zhanxu, checi):
    other_checi_all = df[df['站点']==zhandian]['车次id'].values
    other_checi_all = other_checi_all.tolist()
    if current_checi in other_checi_all:
        other_checi_all.remove(current_checi)
    checi_sequence_checked  = []
    
    for each in other_checi_all:
        this_zhanxu = df[(df['车次id']==each)&(df['站点']==zhandian)]['站序'].values[0]
        if this_zhanxu>start_zhandian_zhanxu:
            checi_sequence_checked.append(each)
            
    return checi_sequence_checked

In [253]:
def run(checi_all):
    final_df= pd.DataFrame(columns=['车次','子车次'])
    for current_checi in checi_all:
        data = shike_df[shike_df['车次id']==current_checi]
        current_zhandian_arr = data['站点'].values
        current_zhandian_time_arr = data['时间'].values
        current_checi_match_res = []

        for each_zhandian_index in range(len(current_zhandian_arr)):
            #起点站点
            each_zhandian_start = current_zhandian_arr[each_zhandian_index]
            other_checi_match,duiying_zhanxu= find_other_checi_with_zhandian(current_checi,each_zhandian_start,shike_df)
            #剩余站点
            zhandian_left = current_zhandian_arr[each_zhandian_index+1:len(current_zhandian_arr)]
            checi_match_2times = []
            for each_left_zhandian in zhandian_left:  
                for c_idx in range(len(other_checi_match)):
                    c = other_checi_match[c_idx]
                    df = shike_df[shike_df['车次id']==c]
                    left_checi_match = find_checi_with_left_zhandian(current_checi,each_left_zhandian,df,duiying_zhanxu[c_idx],c)
                    checi_match_2times += left_checi_match

            current_checi_match_res += checi_match_2times  

        res = list(set(current_checi_match_res))
        final_res = []
        current_zhanci = len(shike_df[shike_df['车次id']==current_checi])

        for each in res:
            if current_zhanci>=len(shike_df[shike_df['车次id']==each]):
                if each!=current_checi:
                    final_res.append(each)
            
        
        final_df.loc[len(final_df.index)] =[current_checi, final_res]
        
    return final_df
    

In [298]:
#检查是否是平峰期
def check_peak(checi,df):
    this_checi_df = df[df['车次id']==checi]
    start_time = this_checi_df.iloc[0]['时间']
    end_time = this_checi_df.iloc[len(this_checi_df)-1]['时间']
    t_start = datetime.strptime(start_time, "%H:%M:%S")
    t_end = datetime.strptime(end_time, "%H:%M:%S")
    nonpeak = False
    
    nonpeak_day_start = datetime.strptime("09:00:00", "%H:%M:%S")
    nonpeak_day_end = datetime.strptime("16:30:00", "%H:%M:%S")
    nonpeak_night_start = datetime.strptime("19:30:00", "%H:%M:%S")
    
    if (t_start>=nonpeak_day_start) & (t_start<=nonpeak_day_end):
        nonpeak = True
    if t_start>nonpeak_night_start:
        nonpeak = True
        
    return nonpeak

In [254]:
final3 = run(shike_df['车次id'].unique())

In [255]:
final3 

,车次,子车次
0,411,[]
1,412,[]
2,413,[]
3,414,[]
4,415,[]
...,...,...
554,128,[42]
555,129,[43]
556,130,[44]
557,131,[45]


In [256]:
final3['子车次数量'] = final3['子车次'].apply(len)

In [290]:
final_result = final3[final3['子车次数量']>0]

In [299]:
final_result['平峰期'] = final_result.apply(lambda row: check_peak(row['车次'],shike_df), axis=1)

C:\Users\10157\AppData\Local\Temp\ipykernel_17432\985332825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result['平峰期'] = final_result.apply(lambda row: check_peak(row['车次'],shike_df), axis=1)


In [292]:
def get_checi_info(checi):
    text = ''
    info_df = shike_df[shike_df['车次id']==checi]
    for index, row in info_df.iterrows():
        zd = row['站点']
        time = row['时间']
        new = zd+"("+time+")"+"->"
        text += new
    text = text[:-2]
    return text   
        

In [293]:
def get_zicheci_info(checi_list):
    text = ''
    for each in checi_list:
        checi_title = "#"+str(each)+"#"
        text+=checi_title
        info_df = shike_df[shike_df['车次id']==each]
        for index, row in info_df.iterrows():
            zd = row['站点']
            time = row['时间']
            new = zd+"("+time+")"+"->"
            text += new
        text = text[:-2]
        text +="     "
    return text   
        

In [294]:
final_result['该车次路线'] = final_result.apply(lambda row: get_checi_info(row['车次']), axis=1)

C:\Users\10157\AppData\Local\Temp\ipykernel_17432\2237346576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result['该车次路线'] = final_result.apply(lambda row: get_checi_info(row['车次']), axis=1)


In [295]:
final_result['子车次路线'] = final_result.apply(lambda row: get_zicheci_info(row['子车次']), axis=1)

C:\Users\10157\AppData\Local\Temp\ipykernel_17432\3155844742.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result['子车次路线'] = final_result.apply(lambda row: get_zicheci_info(row['子车次']), axis=1)


In [300]:
final_result

,车次,子车次,子车次数量,平峰期,该车次路线,子车次路线
7,418,[419],1,False,临江(08:20:00)->8849人才公寓(08:25:00)->云驰街(08:26:00),#419#临江(08:20:00)->云驰街(08:26:00)
11,422,"[421, 335]",2,False,临江(08:30:00)->韦恩(08:35:00)->云驰街(08:36:00),#421#临江(08:30:00)->云驰街(08:36:00) #335#盛景华庭...
12,423,[424],1,False,临江(08:35:00)->8849人才公寓(08:40:00)->韦恩(08:41:00)...,#424#临江(08:35:00)->云驰街(08:41:00)
14,425,"[426, 427]",2,False,临江(08:40:00)->云驰街(08:46:00),#426#临江(08:40:00)->云驰街(08:46:00) #427#临江(0...
15,426,"[425, 427]",2,False,临江(08:40:00)->云驰街(08:46:00),#425#临江(08:40:00)->云驰街(08:46:00) #427#临江(0...
...,...,...,...,...,...,...
554,128,[42],1,True,云驰街(12:00:00)->中科(12:08:00)->张墩路(12:18:00)->雨山...,#42#临江(12:18:00)->云驰街(12:23:00)->网易(12:32:00)-...
555,129,[43],1,True,云驰街(13:00:00)->中科(13:08:00)->张墩路(13:18:00)->雨山...,#43#临江(13:18:00)->云驰街(13:23:00)->网易(13:32:00)-...
556,130,[44],1,True,云驰街(14:00:00)->中科(14:08:00)->张墩路(14:18:00)->雨山...,#44#临江(14:18:00)->云驰街(14:23:00)->网易(14:32:00)-...
557,131,[45],1,True,云驰街(15:00:00)->中科(15:08:00)->张墩路(15:18:00)->雨山...,#45#临江(15:18:00)->云驰街(15:23:00)->网易(15:32:00)-...


In [301]:
final_result.to_excel('C:/intern/查找子路线.xlsx',index=False)


In [196]:
shike_df[shike_df['车次id']==418]

,路线,车次id,站序,站点,时间
2318,Y1,418,1,临江,08:20:00
2319,Y1,418,2,8849人才公寓,08:25:00
2320,Y1,418,3,云驰街,08:26:00


In [197]:
shike_df[shike_df['车次id']==419]

,路线,车次id,站序,站点,时间
2321,Y1,419,1,临江,08:20:00
2322,Y1,419,2,云驰街,08:26:00


In [213]:
a="aaaaaaaaaa \n b"
a

'aaaaaaaaaa \n b'

In [179]:
a

[2, 3]